## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-10-01-23-46-00 +0000,wsj,South Korea’s Inflation Rebounds in September,https://www.wsj.com/economy/south-koreas-infla...
1,2025-10-01-23-42-03 +0000,nyt,The Global Sumud Flotilla Is Approaching Gaza....,https://www.nytimes.com/2025/10/01/world/middl...
2,2025-10-01-23-37-42 +0000,nypost,"House GOP asks RFK Jr., Dr. Oz if Andrew Cuomo...",https://nypost.com/2025/10/01/us-news/house-go...
3,2025-10-01-23-21-00 +0000,wsj,Opinion | Our Friends the al-Thanis of Qatar,https://www.wsj.com/opinion/donald-trump-qatar...
4,2025-10-01-23-19-07 +0000,nypost,Bloods gangbanger linked to NYC hookah lounge ...,https://nypost.com/2025/10/01/us-news/bloods-g...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2310/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
55,trump,41
208,government,37
204,shutdown,36
160,will,20
64,new,17


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
258,2025-10-01-10-47-06 +0000,nypost,Trump taunts Dems with ‘Trump 2028’ hats durin...,https://nypost.com/2025/10/01/us-news/presiden...,170
24,2025-10-01-22-43-00 +0000,wsj,Many GOP senators seeking an end to the govern...,https://www.wsj.com/politics/policy/white-hous...,154
35,2025-10-01-22-10-12 +0000,nypost,Trump admin will keep Statue of Liberty open d...,https://nypost.com/2025/10/01/us-news/trump-ad...,151
355,2025-10-01-00-39-00 +0000,wsj,The government was on course to shut down earl...,https://www.wsj.com/politics/policy/government...,146
221,2025-10-01-13-50-00 +0000,wsj,The White House and lawmakers failed to reach ...,https://www.wsj.com/politics/policy/government...,121


## Select top headlines

In [5]:
top_rows = []
working = word_scores.copy()
remaining = latest.copy()
for _ in range(10):
    ranked_loop = remaining.assign(score=remaining['title'].apply(
        lambda t: sum(working.get(w.lower(), 0)
                      for w in re.findall(r'[A-Za-z]+', t)
                      if len(w) > 1)
    )).sort_values('score', ascending=False)
    if ranked_loop.empty:
        break
    top_story = ranked_loop.iloc[0]
    top_rows.append(top_story[['score','pubdate','source','title','link']])
    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for w in words:
        working.pop(w, None)
    remaining = remaining.drop(top_story.name)
top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
258,170,2025-10-01-10-47-06 +0000,nypost,Trump taunts Dems with ‘Trump 2028’ hats durin...,https://nypost.com/2025/10/01/us-news/presiden...
152,86,2025-10-01-17-43-30 +0000,nypost,What will ‘TrumpRx’ actually mean for patients...,https://nypost.com/2025/10/01/health/what-will...
355,68,2025-10-01-00-39-00 +0000,wsj,The government was on course to shut down earl...,https://www.wsj.com/politics/policy/government...
8,68,2025-10-01-23-07-36 +0000,wapo,Israel steps up Gaza City assault as Hamas wei...,https://www.washingtonpost.com/world/2025/10/0...
188,59,2025-10-01-15-20-56 +0000,latimes,Supreme Court puts off decision on whether Tru...,https://www.latimes.com/politics/story/2025-10...
53,51,2025-10-01-21-26-08 +0000,nypost,"Mikie Sherrill launches new ad blaming Trump, ...",https://nypost.com/2025/10/01/us-news/mikie-sh...
285,49,2025-10-01-10-00-00 +0000,nypost,Howard Lutnick tells ‘Pod Force One’ ex-neighb...,https://nypost.com/2025/10/01/us-news/howard-l...
203,45,2025-10-01-14-35-35 +0000,latimes,Missing California woman found dead behind 'se...,https://www.latimes.com/california/story/2025-...
215,42,2025-10-01-14-05-00 +0000,wsj,White House nominees to lead the BLS and the C...,https://www.wsj.com/politics/policy/white-hous...
275,40,2025-10-01-10-00-00 +0000,latimes,"Jane Fonda, derided as 'Hanoi Jane' and a trai...",https://www.latimes.com/california/story/2025-...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
